In [198]:
import numpy as np
import torch
import torch.nn.functional as F
import matplotlib.pyplot as plt
from torch.autograd import Variable
import pandas as pd


In [199]:
df=pd.read_csv('diabetes.csv',header=None,sep=',',dtype=np.float32)

In [200]:
df.head(10)

,0,1,2,3,4,5,6,7,8
0,-0.294118,0.487437,0.180328,-0.292929,0.000000,0.001490,-0.531170,-0.033333,0.0
1,-0.882353,-0.145729,0.081967,-0.414141,0.000000,-0.207153,-0.766866,-0.666667,1.0
2,-0.058823,0.839196,0.049180,0.000000,0.000000,-0.305514,-0.492741,-0.633333,0.0
3,-0.882353,-0.105528,0.081967,-0.535354,-0.777778,-0.162444,-0.923997,0.000000,1.0
4,0.000000,0.376884,-0.344262,-0.292929,-0.602837,0.284650,0.887276,-0.600000,0.0
5,-0.411765,0.165829,0.213115,0.000000,0.000000,-0.236960,-0.894962,-0.700000,1.0
6,-0.647059,-0.216080,-0.180328,-0.353535,-0.791962,-0.076006,-0.854825,-0.833333,0.0
7,0.176471,0.155779,0.000000,0.000000,0.000000,0.052161,-0.952178,-0.733333,1.0
8,-0.764706,0.979899,0.147541,-0.090909,0.283688,-0.090909,-0.931682,0.066667,0.0
9,-0.058823,0.256281,0.573770,0.000000,0.000000,0.000000,-0.868488,0.100000,0.0


In [201]:
df.columns

Int64Index([0, 1, 2, 3, 4, 5, 6, 7, 8], dtype='int64')

In [202]:
x_data=Variable(torch.from_numpy(df[[0, 1, 2, 3, 4, 5, 6, 7]].as_matrix()))

In [203]:
y_data=Variable(torch.from_numpy(df[[8]].as_matrix()))

In [204]:
x_data.size()

torch.Size([759, 8])

In [205]:
y_data.size()

torch.Size([759, 1])

In [206]:
class Network(torch.nn.Module):
    
    def __init__(self):
        super(Network,self).__init__()
        self.fc1=torch.nn.Linear(8,4)   #dropping layers, was close to having 4fc instead of 2, adding layer did not help
        self.fc2=torch.nn.Linear(4,1)
 
        #self.sigmoid=torch.nn.Sigmoid()
        
        
    def forward(self,x):
        
        out1=F.relu(self.fc1(x))
        out2=F.sigmoid(self.fc2(out1))

        #sof max at end, or sigmoid all the way did not work out
        y_pred=out2
        return y_pred
        

In [195]:
model=Network()

In [207]:
criterion=torch.nn.BCELoss(size_average=True)   #adding weight decay did not help, amsgrad did not help, BCE loss was better than rest
optimizer=torch.optim.Adam(model.parameters(),lr=0.001)  #had to change from SGD to Adam but later noy much difference

In [208]:
epochs=10000

for epoch in range(epochs):
    
    y_pred=model(x_data)
    loss=criterion(y_pred,y_data)
    
    optimizer.zero_grad()
    loss.backward()
    optimizer.step()
    y_pred_round=torch.round(y_pred)
    #mean_acc=torch.mean(y_data==y_pred_round)
    mean_acc=(y_data==y_pred_round).sum().item()/y_data.size()[0]
    if (epoch%100==0): print('epoch',epoch,'/',epochs,'loss:',loss,'Accuracy',mean_acc)
    
    

epoch 0 / 10000 loss: tensor(0.7042) Accuracy 0.40711462450592883
epoch 100 / 10000 loss: tensor(0.6332) Accuracy 0.6561264822134387
epoch 200 / 10000 loss: tensor(0.5762) Accuracy 0.6666666666666666
epoch 300 / 10000 loss: tensor(0.5241) Accuracy 0.7496706192358367
epoch 400 / 10000 loss: tensor(0.4966) Accuracy 0.7707509881422925
epoch 500 / 10000 loss: tensor(0.4816) Accuracy 0.764163372859025
epoch 600 / 10000 loss: tensor(0.4731) Accuracy 0.769433465085639
epoch 700 / 10000 loss: tensor(0.4685) Accuracy 0.7786561264822134
epoch 800 / 10000 loss: tensor(0.4661) Accuracy 0.7760210803689065
epoch 900 / 10000 loss: tensor(0.4647) Accuracy 0.7760210803689065
epoch 1000 / 10000 loss: tensor(0.4638) Accuracy 0.7760210803689065
epoch 1100 / 10000 loss: tensor(0.4630) Accuracy 0.7760210803689065
epoch 1200 / 10000 loss: tensor(0.4623) Accuracy 0.7747035573122529
epoch 1300 / 10000 loss: tensor(0.4617) Accuracy 0.7733860342555995
epoch 1400 / 10000 loss: tensor(0.4612) Accuracy 0.7733860342

In [41]:
y_pred[0:10]

tensor([[ 0.5905],
        [ 0.6681],
        [ 0.5801],
        [ 0.6763],
        [ 0.5667],
        [ 0.6347],
        [ 0.6855],
        [ 0.6124],
        [ 0.5841],
        [ 0.5918]])

In [28]:
y_pred_round[0:10]

tensor([[ 0.],
        [ 1.],
        [ 0.],
        [ 1.],
        [ 0.],
        [ 1.],
        [ 1.],
        [ 1.],
        [ 0.],
        [ 0.]])